In [1]:
import os, time, sys,shutil
import pandas as pd
from digital_land.collection import Collection

from functions import run_endpoint_workflow
from sqlite_query_functions import DatasetSqlite
from convert_functions import convert_resource




/Users/jimmytidey/Projects/jupyter-analysis/venv/lib/python3.10/site-packages/pyproj/transformer.py:207: UserWarning: Best transformation is not available due to missing Grid(short_name=uk_os_OSTN15_NTv2_OSGBtoETRS.tif, full_name=, package_name=, url=https://cdn.proj.org/uk_os_OSTN15_NTv2_OSGBtoETRS.tif, direct_download=True, open_license=True, available=False)
  super().__init__(


In [4]:
def conservation_area_checker(url):
    collection_name = 'listed-building-collection'
    dataset = 'listed-building-outline'
    organisation = 'local-authority-eng:CAM'
    endpoint_url = url
    plugin = None
    additional_column_mappings=None
    additional_concats=None
    data_dir = 'data/endpoint_checker_' + str(int(time.time()*10000)) 

    workflow_state = run_endpoint_workflow(
        collection_name,
        dataset,
        organisation,
        endpoint_url,
        plugin,
        data_dir,
        additional_col_mappings=additional_column_mappings,
        additional_concats=additional_concats
    )
    
    print('workflow_state')
    print(workflow_state)
    if(workflow_state):
        print('conditional run')
        checker_result = 'Data checker failed - ensure you have a valid CSV, GeoJSON, GML, or Geopackage file'
        ret = {'checker_result': checker_result, 'issues': []}
        shutil.rmtree(data_dir)
        return ret    


    collection = Collection(os.path.join(data_dir,'collection'))
    collection.load(directory=os.path.join(data_dir,'collection'))
    logs = collection.log.entries
    logs = pd.DataFrame.from_records(logs)

    http_status = logs['status'].values[0]

    if(http_status != 200): 
        checker_state = "Could not access URL"
        ret = {'checker_result': checker_result, 'issues': []}
        shutil.rmtree(data_dir)
        return ret          

    try: 
        dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
        results = dataset_db.get_issues()
        display(results)
        row_count = results.shape[0]

        if(row_count == 0):
            checker_state = "No rows were processed, check you have created a valid file."
        else:
            checker_state = f"{row_count} rows have been processed"

        issues = results.loc[(results.issue_type != 'default-value')]

    except:
        issues = pd.DataFrame()

   
    ret = {'checker_state': checker_state, 'issues': issues.to_dict('records')}
    
    shutil.rmtree(data_dir)

    return ret


conservation_area_checker('https://raw.githubusercontent.com/jimmytidey/jupyter-analysis/feature-endpoint-checker-api/endpoint_checker/examples/conservation_area_invalid_csv.csv')


HTTP Error 404: Not Found
HTTP Error 404: Not Found
data/endpoint_checker_16967532754843/collection
[]
*No resources collected view collection logs for more info*
workflow_state
No resources collected view collection logs for more info
conditional run


{'checker_result': 'Dat checker failed - ensure you have a valid CSV, GeoJSON, GML, or Geopackage file',
 'issues': []}